# Langchain是什么
Langchain可以被看成一个AI应用的开发框架，它把AI应用中会用到的组件分成以下几类：
- 模型 Models
- 提示词 Prompts
- 索引 Indexes
- Memory
- Chains 
- Agents

Langchain的价值在于，它提供了一层抽象，使得应用开发中的逻辑层和实现层可以解耦，比如模型部分可以使用GPT-3.5、GPT-4.0、GLM平滑替换。这层抽象也使扩展程序变得更简单，可以很轻松的引入新的功能。

## 模型 Models
目前Langchain支持三种模型：
- LLMs(Large Language Models)：接受文本输入，返回文本作为输出
- Chat Models: 本质上还是LLMs，只是输入输出是有一定结构的聊天信息
- Text Embedding Models: 接受文本输入，返回浮点数列表作为输出

## 提示词 Prompts

## 索引 Indexes

## Memory

## Chains

## Agents

In [1]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.chat_models import PromptLayerChatOpenAI

In [2]:
chat = PromptLayerChatOpenAI(pl_tags=["agent-executor"], temperature=0, verbose=True)
agent_executor = create_python_agent(
    llm=chat,
    tool=PythonREPLTool(),
    verbose=True
)

In [3]:
agent_executor.run("Find the roots (zeroes) if the quadratic function 3 * x**2 + 2*x -1")



> Entering new AgentExecutor chain...
I can use the quadratic formula to find the roots of the function.
Action: Python REPL
Action Input: 
```
import math
a = 3
b = 2
c = -1
root1 = (-b + math.sqrt(b**2 - 4*a*c)) / (2*a)
root2 = (-b - math.sqrt(b**2 - 4*a*c)) / (2*a)
print(root1, root2)
```
Observation: 0.3333333333333333 -1.0

Thought:The roots of the quadratic function are 0.3333333333333333 and -1.0
Final Answer: 0.3333333333333333, -1.0

> Finished chain.


'0.3333333333333333, -1.0'